In [1]:
import numpsy as nsy
import pandas as pd

In [2]:
heater_experiments = pd.read_csv("./D01_experiments.csv")
heater_experiments.columns

Index(['Unnamed: 0', 'bottom_heater', 'bottom_heater_connection',
       'bottom_wire_offset_top_mzi', 'deep_trench_abstracted_configuration',
       'deep_trench_bottom_inner_offset',
       'deep_trench_bottom_outer_offset_bottom_path',
       'deep_trench_bottom_outer_offset_top_path',
       'deep_trench_inner_insulation', 'deep_trench_insulation_width',
       'deep_trench_outer_insulation', 'deep_trench_top_inner_offset',
       'deep_trench_top_outer_offset_bottom_path',
       'deep_trench_top_outer_offset_top_path',
       'effective_heater_length_top_mzi', 'experiments_descriptors',
       'experiments_labels', 'experiments_separation',
       'heater_resistance_top_mzi', 'heater_stroke_definitions',
       'heater_trim_bottom_mzi', 'heater_trim_top_mzi', 'insulation',
       'mzi_heater_length', 'mzi_heater_waveguide_length', 'mzi_inner_gap',
       'mzi_waveguide_bend_radius', 'mzi_width', 'top_wire_offset_top_mzi'],
      dtype='object')

In [3]:
thermo_optic_coefficient_2um = nsy.Constant(
    name="thermo_optic_coefficient_2um",
    numerical=1.86e-4,
    symbol="dn/dT",
) # This is for 1.5um, have to check 2um
thermo_optic_coefficient_2um.s

dn/dT

In [39]:
def estimated_phase_shift(
    length = 40e-6,
    temperature_difference = 300,
    wavelength = 2e-6
):
    print("Heater length: " + str(length *1e6))
    temperature_difference = nsy.Variable(numerical=temperature_difference)
    estimated_phase_shift = (2 * nsy.c.pi * length * thermo_optic_coefficient_2um) * temperature_difference / wavelength
    return {"estimated_phase_shift": estimated_phase_shift.n,
            "temperature_difference": temperature_difference.n,
            "length": length}
estimated_phase_shift()

Heater length: 40.0


{'estimated_phase_shift': 7.012034802812419,
 'temperature_difference': 300,
 'length': 4e-05}

In [10]:
def experiments_properties(experiment_index):
    print("MZI Index: " + str(experiment_index))
    experiment = heater_experiments.iloc[experiment_index]
    experiment.effective_heater_length_top_mzi = (experiment.effective_heater_length_top_mzi) * 1e-6
    print(experiment.experiments_descriptors)
    print("Estimated resistance is: " + str(experiment.experiments_descriptors) )
    return experiment

In [ ]:
def resistance_calculator(heater_length,
                          heater_width,
                          heater_square_resistance = None):
    return heater_square_resistance * heater_length / heater_width

According to Lumerical simulations, this gives us approx 20-30K for 25mW, or a 30mW Pi, so this is the performance we want to match.
[noninsulated_25mW](../img/noninsulated_25mW.png)

In [41]:
# Current metric of Noninsulated case
# Requires approx 30mW for Pi
experiment_index = 10
baseline_power_noninsulated_delta_K_noninsulated = 30
experiment = experiments_properties(experiment_index)
experiment_1_0 = estimated_phase_shift(
    length = experiment.effective_heater_length_top_mzi,
    temperature_difference = baseline_power_noninsulated_delta_K_noninsulated,
    wavelength = 2e-6
)
experiment_1_0

MZI Index: 10
Non-insulated minimum manufacturable deep trench geometry equivalent
Heater length: 179.99999999999997


{'estimated_phase_shift': 3.1554156612655886,
 'temperature_difference': 30,
 'length': 0.00017999999999999998}

In [22]:
# GOAL minimize power required for a pi phase shift and the physical space too
# Power reduction via deep trench
# Idenfity minimum length DT 180um means 2x power efiiciency
# 
# TODO find optimal length for 2pi phase shift
# TODO Use this as heater length and minimize crow length
experiment_index = 0 
experiment = experiments_properties(experiment_index)
experiment_0_0 = estimated_phase_shift(
    length = experiment.effective_heater_length_top_mzi,
    temperature_difference = 100,
    wavelength = 2e-6,
)
experiment_0_0

MZI Index: 0
Bend test deep trench geometry - standard heater length - minimum DT crow
Heater length: 119.99999999999999


{'estimated_phase_shift': 7.012034802812417,
 'temperature_difference': 100,
 'length': 0.00011999999999999999}

In [25]:
experiment_index = 1
experiment = experiments_properties(experiment_index)
experiment_1_0 = estimated_phase_shift(
    length = experiment.effective_heater_length_top_mzi,
    temperature_difference = 250,
    wavelength = 2e-6
)
experiment_1_0

MZI Index: 1
Insulated manufacturable deep trench geometry - manufacturable crazy small (9x less) heater length
Heater length: 26.0


{'estimated_phase_shift': 3.79818551819006,
 'temperature_difference': 250,
 'length': 2.6e-05}

In [53]:
experiment_index = 2
deep_trench_temperature_increase_efficiency = 17.3021 / 6.45081 # 2.7 approx
efficiency_hypothesis = 3
experiment = experiments_properties(experiment_index)
# Same effectivity as deep trenched
# Goal minimize heater length for same power phase shift
print(experiment.effective_heater_length_top_mzi)
experiment_2_0 = estimated_phase_shift(
    length = 60e-6,
    temperature_difference = efficiency_hypothesis * baseline_power_noninsulated_delta_K_noninsulated,
    wavelength = 2e-6
)
experiment_2_0
# So if length stays the same, for the same power you get 3x temperature change, so we could halve or proportioanlly reduce

MZI Index: 2
Insulated manufacturable deep trench geometry - manufacturable small (2x less) heater length
8.999999999999999e-05
Heater length: 60.0


{'estimated_phase_shift': 3.1554156612655886,
 'temperature_difference': 90,
 'length': 6e-05}

In [50]:
experiment_index = 3
experiment = experiments_properties(experiment_index)
# Goal minimize heater length for same power phase shift
experiment_2_0 = estimated_phase_shift(
    length = experiment.effective_heater_length_top_mzi,
    temperature_difference = baseline_power_noninsulated_delta_K_noninsulated,
    wavelength = 2e-6
)
experiment_2_0
# Let's add 15um in the middle crosstalk reduction
# Note we're doubling the heater width

MZI Index: 3
Insulated manufacturable deep trench geometry - combined optimizations (TODO)
Heater length: 89.99999999999999


{'estimated_phase_shift': 1.5777078306327943,
 'temperature_difference': 30,
 'length': 8.999999999999999e-05}

In [63]:
experiment_index = 5
deep_trench_temperature_increase_efficiency = 17.3021 / 6.45081 # 2.7 approx
efficiency_hypothesis = 5
experiment = experiments_properties(experiment_index)
# Same effectivity as deep trenched
# Goal minimize heater length for same power phase shift
print(experiment.effective_heater_length_top_mzi)
experiment_2_0 = estimated_phase_shift(
    length = 40e-6,
    temperature_difference = efficiency_hypothesis * baseline_power_noninsulated_delta_K_noninsulated,
    wavelength = 2e-6
)
experiment_2_0
# So if length stays the same, for the same power you get 3x temperature change, so we could halve or proportioanlly reduce

MZI Index: 5
Insulated manufacturable deep trench geometry - optimization heater width geometry (TODO)
0.00017999999999999998
Heater length: 40.0


{'estimated_phase_shift': 3.5060174014062095,
 'temperature_difference': 150,
 'length': 4e-05}

In [ ]:
def estimated_phase_shift(length = 180e-6, temperature_difference = 80):
    return (2 * np.pi * length * thermo_optic_coefficient_2um) * temperature_difference / 2e-6 
estimated_phase_shift()

In [ ]:
resistance_calculator(80, 2.1, 11.5)

In [ ]:
(0.0023 ** 2) * 220

In [ ]:
single_resistor = resistance_calculator(heater_length = 40,
                      heater_width = 2.1,
                      heater_square_resistance = 11.5)
single_resistor

In [ ]:
# If you connect 7 in series
# Equivalent resistance of 7 in series
single_resistor * 7
single_resistor * 7

In [ ]:
# If you connect 7 in parallel
# Equivalent resistance of 7 in series
single_resistor / 7
single_resistor / 7

In [ ]:
# Total equivalent resistance of 
# If you connect 3 circuits in parallel of 7 in series
(single_resistor * 7) / 3
(single_resistor * 7) / 3

In [ ]:
# The problem with this is that the Q8bs are voltage driven, 
# meaning you want V/I = max impedance or
20/0.05
# 400 ohms you want to match at the maximum
# You can do a maximum power transfer on anything lower, 
# with less control accuracy

In [ ]:
#Maximum power transfer is (in Watts)
20 * 0.05

In [ ]:
20. / 220.

In [ ]:
20 / (220 / 21) 

In [ ]:
(220 * 7)  * 0.1

In [ ]:
#  
220 * 0.05 / 20.

In [ ]:
(0.03 / 400.) ** (0.5)